In [25]:
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [26]:
# 情绪分析
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [27]:
# 将文本输入转换为向量
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [28]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
 
# transformers 模型的适量输出非常大，包含三个维度：
# batch_size（批量大小）, sequence_length（序列长度）, hidden_size（隐藏层大小）

outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


以情感分类为例，我们需要一个带有序列分类头的模型（能够将句子分类为积极或消极）。因此，我们不选用 AutoModel 类，而是使用 AutoModelForSequenceClassification 。也就是说前面写的 model = AutoModel.from_pretrained(checkpoint) 并不能得到情感分类任务的结果，因为没有加载 Model head。 AutoModelForSequenceClassification 类在 AutoModel 的基础上添加了一个序列分类头部，可以 将文本分类为不同的类别。

In [29]:
import torch
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

print(outputs.logits.shape)

print("维度会降低很多：模型头接收我们之前看到的高维向量作为输入，并输出包含两个值（每种标签一个）的向量。")

print("----------")
print(outputs.logits)

# 这些不是概率，而是 logits（对数几率） ，是模型最后一层输出的原始的、未标准化的分数。要转换为概率，
# 它们需要经过 SoftMax 层（所有🤗Transformers 模型的输出都是 logits，因为训练时的损失函数通常会将最后的激活函数（如 SoftMax）与实际的损失函数（如交叉熵）融合）：

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

print("查看对应的标签：")
print(model.config.id2label)

torch.Size([2, 2])
维度会降低很多：模型头接收我们之前看到的高维向量作为输入，并输出包含两个值（每种标签一个）的向量。
----------
tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)
查看对应的标签：
{0: 'NEGATIVE', 1: 'POSITIVE'}


### 创建一个 Transformer 模型

In [30]:
from transformers import BertConfig, BertModel

# 初始化 Config 类
config = BertConfig()

# 从 Config 类初始化模型
model = BertModel(config)

print(config)

# 这个模型是可以运行并得到结果的，但它会输出胡言乱语；它需要先进行训练才能正常使用。

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



### Tokenizer

In [31]:
# 基于 单词 的 Tokenizer
print("----基于 单词 的 Tokenizer----")
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

# 基于 字符 的 Tokenizer
print("----基于 字符 的 Tokenizer----")
tokenized_text = list("Jim Henson was a puppeteer")
print(tokenized_text)

# 基于 子词 的 Tokenizer
print("----基于 子词 的 Tokenizer----")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_text = tokenizer("Jim Henson was a puppeteer")
print(tokenized_text)


----基于 单词 的 Tokenizer----
['Jim', 'Henson', 'was', 'a', 'puppeteer']
----基于 字符 的 Tokenizer----
['J', 'i', 'm', ' ', 'H', 'e', 'n', 's', 'o', 'n', ' ', 'w', 'a', 's', ' ', 'a', ' ', 'p', 'u', 'p', 'p', 'e', 't', 'e', 'e', 'r']
----基于 子词 的 Tokenizer----
{'input_ids': [101, 3104, 1124, 15703, 1108, 170, 16797, 8284, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


将文本翻译成数字被称为编码（encoding）。编码分两步完成：分词，然后转换为 inputs ID。

In [32]:
# tokenization -> tokenization 过程由 tokenizer 的 tokenize() 方法实现：
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [33]:
# 从 tokens 到 inputs ID -> inputs ID 的转换由 tokenizer 的 convert_tokens_to_ids() 方法实现：
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [34]:
# 解码 -> 解码（Decoding） 正好相反：从 inputs ID 到一个字符串。这以通过 decode() 方法实现：
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

print("请注意， decode 方法不仅将索引转换回 tokens，还将属于相同单词的 tokens 组合在一起以生成可读的句子。")

Using a transformer network is simple
请注意， decode 方法不仅将索引转换回 tokens，还将属于相同单词的 tokens 组合在一起以生成可读的句子。


### 处理多个输入

In [35]:
# Transformers 模型默认情况下需要一个句子列表。
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1 = "I've been waiting for a HuggingFace course my whole life."
sequence2 = ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much."]

tokens1 = tokenizer.tokenize(sequence1)
tokens2 = tokenizer.tokenize(sequence2)
ids1 = tokenizer.convert_tokens_to_ids(tokens1)
ids2 = tokenizer.convert_tokens_to_ids(tokens2)

input_ids1 = torch.tensor([ids1])
input_ids2 = torch.tensor([ids2])
print("Input IDs1:", input_ids1)
print("Input IDs2:", input_ids2)

output1 = model(input_ids1)
output2 = model(input_ids2)

print("Logits1:", output1.logits)
print("Logits2:", output2.logits)

print(" tokenizer 不仅仅是将 inputs ID 的列表转换为张量，它还在其上添加了一个维度")

Input IDs1: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Input IDs2: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012,  1045,  5223,  2023,  2061,  2172,  1012]])
Logits1: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
Logits2: tensor([[ 1.7446, -1.4520]], grad_fn=<AddmmBackward0>)
 tokenizer 不仅仅是将 inputs ID 的列表转换为张量，它还在其上添加了一个维度


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [37]:
# 批处理（Batching）是一次性通过模型发送多个句子的行为。如果你只有一句话，你可以构建一个只有一个句子的 batch：
batched_ids = [ids1, ids2]

print("Batched IDs:", batched_ids)


Batched IDs: [[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 1045, 5223, 2023, 2061, 2172, 1012]]


In [38]:
# 填充输入（Padding）
# 如果你的 batch 中的句子长度不同，你需要将它们填充到相同的长度。你可以使用 tokenizer 的 padding 方法：
batched_ids = [ids1, ids2]
print(len(batched_ids[0]), len(batched_ids[1]))

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

# 大多数情况下，您应该为模型提供attention_mask以忽略填充标记，以避免此无提示错误。

14 20
tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


咦，我们批处理预测中的 logits 值有点问题：第二行应该与第二句的 logits 相同，但我们得到了完全不同的值！

这是因为 Transformer 模型的关键特性：注意力层，它考虑了每个 token 的上下文信息。这具体来说，每个 token 的含义并非单独存在的，它的含义还取决于它在句子中的位置以及周围的其他 tokens。当我们使用填充（padding）来处理长度不同的句子时，我们会添加特殊的“填充 token”来使所有句子达到相同的长度。但是，注意力层会将这些填充 token 也纳入考虑，因为它们会关注序列中的所有 tokens。这就导致了一个问题：尽管填充 token 本身并没有实际的含义，但它们的存在会影响模型对句子的理解。我们需要告诉这些注意层忽略填充 token。这是通过使用注意力掩码（attention mask）层来实现的。

In [39]:
# 现在，第二个序列的输出与其实际输出匹配：
attention_mask = torch.tensor([[1, 1, 1], [1, 1, 0]])
output = model(torch.tensor(batched_ids), attention_mask=attention_mask)
print(output.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


如果提供了填充令牌，Transformers 不会自动创建一个 attention_mask 来屏蔽填充令牌，因为：
- 某些模型没有 padding token。
- 对于某些用例，用户希望模型能够处理 padding token。

In [40]:
# 更长的句子
# 此问题有两种解决方案：
# - 使用支持更长序列长度的模型。
# - 截断你的序列。

# 另外，我们建议你通过设定 max_sequence_length 参数来截断序列：
# sequence = sequence[:max_sequence_length]

### 综合应用


In [47]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
sequence = "I've been waiting for a HuggingFace course my whole life."

# model_inputs = tokenizer(sequences)
model_inputs = tokenizer(sequence)

* 多种填充方法

```python
    # 将句子序列填充到最长句子的长度
    model_inputs = tokenizer(sequences, padding="longest")

    # 将句子序列填充到模型的最大长度
    # (512 for BERT or DistilBERT)
    model_inputs = tokenizer(sequences, padding="max_length")

    # 将句子序列填充到指定的最大长度
    model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
```
* 可以对序列进行截断

```python
    model_inputs = tokenizer(sequences, truncation=True)
```

In [48]:
model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [50]:
# 可以看到这两次的输出并不相同
# 分别解码
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

print("tokenizer 在开头添加了特殊单词 [CLS] ，在结尾添加了特殊单词 [SEP] 。这是因为模型在预训练时使用了这些字词，所以为了得到相同的推断结果，我们也需要添加它们。")

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.
tokenizer 在开头添加了特殊单词 [CLS] ，在结尾添加了特殊单词 [SEP] 。这是因为模型在预训练时使用了这些字词，所以为了得到相同的推断结果，我们也需要添加它们。


### 小结：从 tokenizer 到模型

In [52]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

print(output.logits)

tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)
